## Mercari Price Suggestion Challenge

The objective of this challenge is to build an algorithm that automatically suggests the right product prices on Mercari. The training data consists of user-inputted text descriptions of their products, including details like product category name, brand name, and item condition

In [27]:
# Import required libraries
import numpy as np
import pandas as pd

# Text mining 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

# Time 
from time import time

## Exploratory Data Analysis

In [6]:
def startTime():
    return time()
def endTime(s):
    print "Time elapsed {}".format(s-time())


In [19]:
train_df = pd.read_csv('../../data/train.tsv', sep='\t')
#test = pd.read_csv('../../data/test.tsv', sep='\t')

In [20]:
# size of training and dataset
print(train_df.shape)

(593376, 8)


In [21]:
# different data types in the dataset: categorical (strings) and numeric
train_df.dtypes


train_id               int64
name                  object
item_condition_id      int64
category_name         object
brand_name            object
price                float64
shipping               int64
item_description      object
dtype: object

In [22]:
# Overall summary of train data
train_df.describe()

,train_id,item_condition_id,price,shipping
count,593376.000000,593376.000000,593376.000000,593376.000000
mean,296687.500000,1.907738,26.689003,0.446294
std,171293.041002,0.902921,38.340061,0.497108
min,0.000000,1.000000,0.000000,0.000000
25%,148343.750000,1.000000,10.000000,0.000000
50%,296687.500000,2.000000,17.000000,0.000000
75%,445031.250000,3.000000,29.000000,1.000000
max,593375.000000,5.000000,2000.000000,1.000000


In [24]:
# First few rows of the dataset 
train_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [29]:
# Retain only part of the data 
n_samples = 1000
train_df = train_df.iloc[:n_samples,:]
print(train_df.shape)

(1000, 8)


In [32]:
n_features = 1000
n_components = 10
n_top_words = 20


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=5,
                                   max_features=n_features,
                                   stop_words='english')

t0 = startTime()
tfidf = tfidf_vectorizer.fit_transform(train_df)
endTime(t0)

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = startTime()
tf = tf_vectorizer.fit_transform(train_df)
endTime(t0)

Extracting tf-idf features for NMF...


ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.